# 1

In [10]:
# Permuted Choice 1 (64 → 56 bits)
PC1 = [
     57, 49, 41, 33, 25, 17,  9,
      1, 58, 50, 42, 34, 26, 18,
     10,  2, 59, 51, 43, 35, 27,
     19, 11,  3, 60, 52, 44, 36,
     63, 55, 47, 39, 31, 23, 15,
      7, 62, 54, 46, 38, 30, 22,
     14,  6, 61, 53, 45, 37, 29,
     21, 13,  5, 28, 20, 12,  4
]

# Left-shift schedule for the 16 rounds
LSHIFTS = [1, 1, 2, 2, 2, 2, 2, 2,
           1, 2, 2, 2, 2, 2, 2, 1]

# Permuted Choice 2 (56 → 48 bits)
PC2 = [
     14, 17, 11, 24,  1,  5,
      3, 28, 15,  6, 21, 10,
     23, 19, 12,  4, 26,  8,
     16,  7, 27, 20, 13,  2,
     41, 52, 31, 37, 47, 55,
     30, 40, 51, 45, 33, 48,
     44, 49, 39, 56, 34, 53,
     46, 42, 50, 36, 29, 32
]

def permute(block, table, block_len):
    out = 0
    for position in table:
        out <<= 1
        # Convert 1-based position to 0-based index from the MSB side
        shift_amt = block_len - position
        out |= (block >> shift_amt) & 1
    return out

def left_rotate(value: int, bits: int, width: int = 28) -> int:
    return ((value << bits) & ((1 << width) - 1)) | (value >> (width - bits))

def key_schedule(hex_key: str) -> list:
    # 1. Convert the 64-bit hex key to an int
    key64 = int(hex_key, 16) & ((1 << 64) - 1)

    # 2. PC-1: drop parity bits → 56-bit result
    key56 = permute(key64, PC1, 64)

    # Split into C and D (28 bits each)
    C = (key56 >> 28) & ((1 << 28) - 1)
    D =  key56        & ((1 << 28) - 1)

    subkeys = []
    for round_no in range(16):
        # 3. Left-rotate C and D
        shift = LSHIFTS[round_no]
        C = left_rotate(C, shift, 28)
        D = left_rotate(D, shift, 28)

        # 4. Combine and apply PC-2 → 48-bit sub-key
        CD = (C << 28) | D
        Ki = permute(CD, PC2, 56)
        subkeys.append(Ki)

    return subkeys

key_hex = "0101010101010101"   # K[0]
subkeys = key_schedule(key_hex)

print("Round Sub-key (hex)")
print("--------------------")
for i, k in enumerate(subkeys, 1):
        # 48 bits → 12 hex digits
        print(f"{i:>3}    {k:012X}")

Round  Sub-key (hex)
--------------------
  1    000000000000
  2    000000000000
  3    000000000000
  4    000000000000
  5    000000000000
  6    000000000000
  7    000000000000
  8    000000000000
  9    000000000000
 10    000000000000
 11    000000000000
 12    000000000000
 13    000000000000
 14    000000000000
 15    000000000000
 16    000000000000


# 2

In [15]:
SBOX = [
 0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76,
 0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0,
 0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15,
 0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75,
 0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84,
 0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf,
 0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8,
 0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2,
 0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73,
 0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb,
 0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79,
 0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08,
 0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a,
 0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e,
 0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf,
 0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16
]

# RCON[i] is (x^(i-1),00,00,00) in GF(2^8), i = 1 … 10   (only first byte is non-zero)
RCON = [0x00,
        0x01,0x02,0x04,0x08,0x10,
        0x20,0x40,0x80,0x1B,0x36]

def rot_word(word):
    # Rotate 4-byte word left by one byte.
    return word[1:] + word[:1]

def sub_word(word):
    # Apply AES S-box on each byte of the 4-byte word.
    return bytes(SBOX[b] for b in word)

def xor_words(w1, w2):
    # XOR two 4-byte words.
    return bytes(a ^ b for a, b in zip(w1, w2))

def expand_key(key_hex):
    # Convert the 32-hex-char string to 16-byte key
    key_bytes = bytes.fromhex(key_hex)
    
    # W will hold 44 words (4 bytes each)
    W = [key_bytes[i:i+4] for i in range(0, 16, 4)]

    for i in range(4, 44):
        temp = W[i-1]
        if i % 4 == 0:
            temp = xor_words(sub_word(rot_word(temp)), bytes([RCON[i//4], 0x00, 0x00, 0x00]))
        W.append(xor_words(W[i-4], temp))

    # Pack words into round keys
    round_keys = [b''.join(W[4*r:4*(r+1)]) for r in range(11)]  # K0 … K10
    return round_keys

key_hex = "39ef7c5e43b4f9ce5ef3373c3bd264b6"

keys = expand_key(key_hex)
print("Round Key (hex)")
print("-------------------------------")
for r, k in enumerate(keys):
    print(f"{r:>3}  {k.hex()}")

Round Key (hex)
-------------------------------
  0  39ef7c5e43b4f9ce5ef3373c3bd264b6
  1  8dac32bcce18cb7290ebfc4eab3998f8
  2  9dea73de53f2b8acc31944e26820dc1a
  3  2e6cd19b7d9e6937be872dd5d6a7f1cf
  4  7acd5b6d0753325ab9d41f8f6f73ee40
  5  e5e552c5e2b6609f5b627f1034119150
  6  476401dda5d26142feb01e52caa18f02
  7  351776a990c517eb6e7509b9a4d486bb
  8  fd539ce06d968b0b03e382b2a7370409
  9  7ca19dbc113716b712d49405b5e3900c
 10  5bc163694af675de5822e1dbedc171d7


# 3
## Which keys are weak and what makes them so?
A weak key is a key that causes the DES round subkeys to be identical (or to repeat in a predictable way) for all 16 rounds. When this happens the encryption operation becomes its own inverse. This makes the cipher trivially breakable because an attacker can recover the plaintext by simply encrypting the ciphertext again with the same key.

There are 4 weak keys:
```
0x0101010101010101
0xFEFEFEFEFEFEFEFE
0x1F1F1F1F0E0E0E0E
0xE0E0E0E0F1F1F1F1
```
After PC-1 the first two give C0 = D0 = 0x0000000 (all 0’s) or C0 = D0 = 0x0FFFFFF (all 1’s).

Rotating an all-zero or all-one word does nothing, so Ci = C0 and Di = D0
for every i, hence Ki is the same in every round.

After PC-1 the second two give C0 = D0 = 0x0F0F0F0 or 0xF0F0F0F, a pattern
whose 1-bit and 2-bit left rotations reproduce the same value and every sub-key is identical.

Because the 16 sub-keys are identical, encryption and decryption with any of
these four keys are the same operation

```bash
# Using the example command
# Encrypt with a weak DES key (hex 0101010101010101)
openssl enc -nopad -e -des-ecb -K 0101010101010101 -in msg.txt -out cipher.bin -provider legacy

# Encrypt the ciphertext again with the same weak key
openssl enc -nopad -e -des-ecb -K 0101010101010101 -in cipher.bin -out cipher2.bin -provider legacy

# Compare the double‑encrypted file with the original plaintext
diff msg.txt cipher2.bin
# returns 0, where the files are the same proving decryption
```


# 4

## 4.1 OFB Mode  

In OFB mode the block-cipher is used only to generate a keystream; the
plaintext itself is never given to the block-cipher.  
For a block index $i=1,\dots ,n$:

$$
\begin{aligned}
r_0 &= \text{IV},\\[2pt]
r_i &= E_K\!\bigl(r_{\,i-1}\bigr),\\[4pt]
y_i &= x_i \oplus r_i .
\end{aligned}
$$

Encrypting another message $X'=(x'_1,\dots ,x'_n)$ with the same key and
the same IV yields the same keystream $\{r_i\}$:

$$
y'_i = x'_i \oplus r_i .
$$

Hence every ciphertext block pair satisfies  

$$
\boxed{y_i \oplus y'_i \;=\; (x_i\oplus r_i)\oplus(x'_i\oplus r_i)
      \;=\; x_i \oplus x'_i } .
$$

Therefore an eavesdropper who XORs the two ciphertexts obtains the
whole vector $X \oplus X'$ without needing to know the key.

---

## 4.2 CBC Mode  

For CBC mode with a fixed key $K$ and IV we have

$$
\begin{aligned}
c_0 &= \text{IV},\\[2pt]
c_i &= E_K\!\bigl(x_i \oplus c_{\,i-1}\bigr),\qquad i=1,\dots ,n,\\[4pt]
x_i &= D_K(c_i)\; \oplus\; c_{\,i-1}\quad\text{(during decryption).}
\end{aligned}
$$

Consider two plaintext blocks $x_i$ and $x_j$ (possibly in different
messages, provided the same IV was reused).  
Using only ciphertext we have

$$
\begin{aligned}
x_i \oplus x_j
 &= \bigl(D_K(c_i)\oplus c_{\,i-1}\bigr)\;\oplus\;
    \bigl(D_K(c_j)\oplus c_{\,j-1}\bigr) \\[4pt]
 &= D_K(c_i) \oplus D_K(c_j) \oplus c_{\,i-1} \oplus c_{\,j-1}.
\end{aligned}
$$

The terms $D_K(c_i)$ and $D_K(c_j)$ are unknown to the attacker unless
a ciphertext collision occurs:

### Collision Condition  

$$
\boxed{c_i = c_j}.
$$

Because the block-cipher $E_K$ is a permutation,

$$
c_i = c_j \;\Longrightarrow\;
x_i \oplus c_{\,i-1} = x_j \oplus c_{\,j-1}.
$$

Substituting into (above) eliminates the key-dependent terms and yields

$$
\boxed{x_i \oplus x_j = c_{\,i-1} \oplus c_{\,j-1}},
$$

which is computable solely from the observed ciphertext.


# 5
## 5.1 ECB

Encryption     $y_j = E_K(x_j)$

Decryption     $\tilde x_j = D_K(\tilde y_j)$

Because every block is processed independently,

$$
\tilde x_j=
\begin{cases}
x_j, & j\neq i,\\[4pt]
D_K(\tilde y_i)\neq x_i, & j=i.
\end{cases}
$$

Only one plain-text block is affected.

---

## 5.2 OFB 

Keystream     $r_0=\text{IV},\;\; r_j = E_K(r_{j-1})$

Encryption     $y_j = x_j \oplus r_j$

Decryption     $\tilde x_j = \tilde y_j \oplus r_j$

The keystream $\{r_j\}$ is independent of the ciphertext, therefore

$$
\tilde x_j=
\begin{cases}
x_j, & j\neq i,\\[4pt]
x_i \oplus e, & j=i,
\end{cases}
\qquad
e = \tilde y_i \oplus y_i .
$$

Only one plain-text block is corrupted.

---

## 5.3 CBC 

Encryption     $c_0=\text{IV},\;\; c_j=E_K(x_j\oplus c_{j-1})$

Decryption     $x_j = D_K(c_j)\oplus c_{j-1}$

After corruption $\tilde c_i=\tilde y_i$

$$
\tilde x_j=
\begin{cases}
x_j, & j<i,\\[4pt]
D_K(\tilde c_i)\oplus c_{i-1}\;\neq x_i, & j=i,\\[6pt]
D_K(c_{i+1})\oplus \tilde c_i = x_{i+1}\oplus (\tilde c_i\oplus c_i)\neq x_{i+1}, & j=i+1,\\[10pt]
x_j, & j\ge i+2 .
\end{cases}
$$

Exactly two consecutive blocks $(x_i,x_{i+1})$ are wrong.

---

## 5.4 CFB

Register     $s_0=\text{IV},\;\; s_j=E_K(c_{j-1})$

Encryption     $c_j = x_j \oplus s_j$

Decryption     $x_j = c_j \oplus s_j = c_j \oplus E_K(c_{j-1})$

With $\tilde c_i$ we get

$$
\tilde x_j=
\begin{cases}
x_j, & j<i,\\[4pt]
\tilde c_i \oplus E_K(c_{i-1})\neq x_i, & j=i,\\[6pt]
c_{i+1}\oplus E_K(\tilde c_i)
      = x_{i+1}\oplus (\tilde c_i\oplus c_i) \neq x_{i+1},& j=i+1,\\[10pt]
x_j, & j\ge i+2 .
\end{cases}
$$

Exactly two plain-text blocks are corrupted.